In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("train.csv")

# df.replace("?", np.nan, inplace=True)
df.sort_index()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [3]:
print(df.info())
print(df.describe(include="all"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

| Feature | Transformation |
| --- | ----------- |
| PassengerId | |
| HomePlanet | |
| CryoSleep | |
| Cabin | |
| Destination | |
| Age | |
| VIP | |
| RoomService | |
| FoodCourt | |
| ShoppingMall | |
| Spa | |
| VRDeck | |
| Name | |
| Deck | |
| CabinNumber | |
| Port | |

In [4]:
numeric_features = ["AB", "AF", "AH", "AM", "AR", "AX", "AY", "AZ", "BC", "BN", "BP", "BQ", "BR", "BZ", 
                    "CB", "CC", "CF", "CH", "CL", "CR", "CS", "CU", "DA", "DE", "DF", "DH", "DI", 
                    "DL", "DN", "DU", "DV", "DY", "EB", "EE", "EG", "EH", "EL", "EP", "EU", "FC", "FE", 
                    "FI", "FL", "FR", "FS", "GB", "GE", "GF", "GH", "GI", "GL"]
binary_features = ["EJ"]
passthrough_features = []
drop_features = []

target = "Class"

In [5]:
preprocessor = make_column_transformer(
    (
        make_pipeline(SimpleImputer(strategy="median"), StandardScaler()),
        numeric_features
    ),
    (
        make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False, dtype=int)),
        binary_features
    ),
    (
        "passthrough",
        passthrough_features
    ),
    (
        "drop",
        drop_features
    )
)

X_train, y_train = df.drop(columns=[target]), df[target]
X_train_transformed = preprocessor.fit_transform(X_train)
pd.DataFrame(X_train_transformed)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,0.339209,...,0.162821,-0.035806,-0.250437,-0.940094,-0.410260,-0.655511,-0.948991,0.531241,-0.814049,1.0
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,-0.575174,...,-0.457925,-0.060566,0.113640,-1.145070,-0.410260,0.687893,-0.238862,-0.509218,1.304860,0.0
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,1.456789,...,0.198663,-0.051023,0.597343,1.637944,-0.299210,-0.051850,-0.351743,-0.424754,-0.806623,1.0
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,0.644004,...,0.060364,-0.060566,-0.104806,-0.219883,-0.342195,-0.650833,0.858232,1.101332,-0.810609,1.0
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.745602,...,0.237302,0.896815,-0.229632,-0.432313,0.099920,-0.318309,1.409422,-0.395228,-0.816349,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,-0.699975,-0.161828,0.040232,-0.422762,0.275215,-0.802577,0.040875,-0.464891,-0.080610,-0.067183,...,-0.457925,-0.045332,-0.271241,-1.177680,0.593022,-0.340465,-0.694915,0.513497,1.304860,0.0
613,-0.088253,0.852755,-0.261669,0.108831,0.556117,0.170319,-0.082686,0.473063,-0.065661,1.659985,...,0.417655,-0.045702,0.004417,1.519617,2.535523,-0.599582,-0.186764,2.048314,-0.811625,1.0
614,-0.106514,-0.453742,0.090140,0.235206,-0.011673,0.990330,-0.083417,1.113828,-0.104787,-0.270379,...,-0.457925,-0.060566,-0.271241,-0.076400,-0.019561,-0.424307,-0.540129,1.903449,1.304860,0.0
615,-0.243466,-0.973904,-0.261669,-0.219353,-0.189295,0.955584,-0.083417,-0.699741,-0.104787,-0.676772,...,0.333481,-0.054771,0.191657,0.387591,-0.410260,-0.657500,-0.646592,-0.370599,-0.807817,1.0


In [6]:
X_test = pd.read_csv("test.csv")

model = make_pipeline(preprocessor, SVC())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [7]:
# Dummy Submission (all false)
dummy_submission_df = pd.DataFrame()
dummy_submission_df["Id"] = X_test["Id"]
dummy_submission_df[target] = False
dummy_submission_df.to_csv("dummy_submission.csv", index=False)

# SVC Submission
svc_submission_df = pd.DataFrame()
svc_submission_df["Id"] = X_test["Id"]
svc_submission_df[target] = y_pred
svc_submission_df.to_csv("svc_submission.csv", index=False)